In [1]:
#### Libraries
# Standard library
import pickle
import gzip
# Third-party libraries
import numpy as np
import theano
import theano.tensor as T
from theano.tensor.nnet import conv
from theano.tensor.nnet import softmax
from theano.tensor import shared_randomstreams
from theano.tensor.signal import pool

# Activation functions for neurons
def linear(z): return z
def ReLU(z): return T.maximum(0.0, z)
from theano.tensor.nnet import sigmoid
from theano.tensor import tanh

In [2]:
#### Constants
GPU = True       #此时计算机为GPU
if GPU:
 print ("Trying to run under a GPU.  If this is not desired, then modify "+\
        "network3.py\nto set the GPU flag to False.")
 try: theano.config.device = 'gpu'
 except: pass # it's already set
 theano.config.floatX = 'float32'
else:
    print ("Running with a CPU.  If this is not desired, then the modify "+\
        "network3.py to set\nthe GPU flag to True.")


Trying to run under a GPU.  If this is not desired, then modify network3.py
to set the GPU flag to False.


In [3]:
#### Load the MNIST data
def load_data_shared(filename="/home/keqingtian/neural-networks-and-deep-learning/data/mnist.pkl.gz"):
    f = gzip.open(filename, 'rb')
    training_data, validation_data, test_data = pickle.load(f,encoding="bytes")
    f.close()
    def shared(data):
        """Place the data into shared variables.  This allows Theano to copy
        the data to the GPU, if one is available.
        """
        shared_x = theano.shared(
            np.asarray(data[0], dtype=theano.config.floatX), borrow=True)
        shared_y = theano.shared(
            np.asarray(data[1], dtype=theano.config.floatX), borrow=True)
        return shared_x, T.cast(shared_y, "int32")
    return [shared(training_data), shared(validation_data), shared(test_data)]

In [4]:
#### Main class used to construct and train networks
class Network(object):

    def __init__(self, layers, mini_batch_size):
        """Takes a list of `layers`, describing the network architecture, and
        a value for the `mini_batch_size` to be used during training
        by stochastic gradient descent.

        """
        self.layers = layers
        self.mini_batch_size = mini_batch_size
        self.params = [param for layer in self.layers for param in layer.params]
        self.x = T.matrix("x")
        self.y = T.ivector("y")
        init_layer = self.layers[0]
        init_layer.set_inpt(self.x, self.x, self.mini_batch_size)
        for j in range(1, len(self.layers)):
            prev_layer, layer  = self.layers[j-1], self.layers[j]
            layer.set_inpt(
                prev_layer.output, prev_layer.output_dropout, self.mini_batch_size)
        self.output = self.layers[-1].output
        self.output_dropout = self.layers[-1].output_dropout

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            validation_data, test_data, lmbda=0.0):
        """Train the network using mini-batch stochastic gradient descent."""
        training_x, training_y = training_data
        validation_x, validation_y = validation_data
        test_x, test_y = test_data

        # compute number of minibatches for training, validation and testing
        num_training_batches = size(training_data)//mini_batch_size     #源代码这里没有用地板除法，得到的是浮点数
        num_validation_batches = size(validation_data)//mini_batch_size
        num_test_batches = size(test_data)//mini_batch_size

        # define the (regularized) cost function, symbolic gradients, and updates
        l2_norm_squared = sum([(layer.w**2).sum() for layer in self.layers])
        cost = self.layers[-1].cost(self)+\
               0.5*lmbda*l2_norm_squared/num_training_batches
        grads = T.grad(cost, self.params)
        updates = [(param, param-eta*grad)
                   for param, grad in zip(self.params, grads)]

        # define functions to train a mini-batch, and to compute the
        # accuracy in validation and test mini-batches.
        i = T.lscalar() # mini-batch index
        train_mb = theano.function(
            [i], cost, updates=updates,
            givens={
                self.x:
                training_x[i*self.mini_batch_size: (i+1)*self.mini_batch_size],
                self.y:
                training_y[i*self.mini_batch_size: (i+1)*self.mini_batch_size]
            })
        validate_mb_accuracy = theano.function(
            [i], self.layers[-1].accuracy(self.y),
            givens={
                self.x:
                validation_x[i*self.mini_batch_size: (i+1)*self.mini_batch_size],
                self.y:
                validation_y[i*self.mini_batch_size: (i+1)*self.mini_batch_size]
            })
        test_mb_accuracy = theano.function(
            [i], self.layers[-1].accuracy(self.y),
            givens={
                self.x:
                test_x[i*self.mini_batch_size: (i+1)*self.mini_batch_size],
                self.y:
                test_y[i*self.mini_batch_size: (i+1)*self.mini_batch_size]
            })
        self.test_mb_predictions = theano.function(
            [i], self.layers[-1].y_out,
            givens={
                self.x:
                test_x[i*self.mini_batch_size: (i+1)*self.mini_batch_size]
            })
        # Do the actual training
        best_validation_accuracy = 0.0
        for epoch in range(epochs):
            for minibatch_index in range(num_training_batches):
                iteration = num_training_batches*epoch+minibatch_index
                if iteration % 1000 == 0:
                    print("Training mini-batch number {0}".format(iteration))
                cost_ij = train_mb(minibatch_index)
                if (iteration+1) % num_training_batches == 0:
                    validation_accuracy = np.mean(
                        [validate_mb_accuracy(j) for j in range(num_validation_batches)])
                    print("Epoch {0}: validation accuracy {1:.2%}".format(
                        epoch, validation_accuracy))
                    if validation_accuracy >= best_validation_accuracy:
                        print("This is the best validation accuracy to date.")
                        best_validation_accuracy = validation_accuracy
                        best_iteration = iteration
                        if test_data:
                            test_accuracy = np.mean(
                                [test_mb_accuracy(j) for j in range(num_test_batches)])
                            print('The corresponding test accuracy is {0:.2%}'.format(
                                test_accuracy))
        print("Finished training network.")
        print("Best validation accuracy of {0:.2%} obtained at iteration {1}".format(
            best_validation_accuracy, best_iteration))
        print("Corresponding test accuracy of {0:.2%}".format(test_accuracy))


In [5]:
#### Define layer types

class ConvPoolLayer(object):
    """Used to create a combination of a convolutional and a max-pooling
    layer.  A more sophisticated implementation would separate the
    two, but for our purposes we'll always use them together, and it
    simplifies the code, so it makes sense to combine them.

    """

    def __init__(self, filter_shape, image_shape, poolsize=(2, 2),
                 activation_fn=sigmoid):
        """`filter_shape` is a tuple of length 4, whose entries are the number
        of filters, the number of input feature maps, the filter height, and the
        filter width.

        `image_shape` is a tuple of length 4, whose entries are the
        mini-batch size, the number of input feature maps, the image
        height, and the image width.

        `poolsize` is a tuple of length 2, whose entries are the y and
        x pooling sizes.

        """
        self.filter_shape = filter_shape
        self.image_shape = image_shape
        self.poolsize = poolsize
        self.activation_fn=activation_fn
        # initialize weights and biases
        n_out = (filter_shape[0]*np.prod(filter_shape[2:])/np.prod(poolsize))
        self.w = theano.shared(
            np.asarray(
                np.random.normal(loc=0, scale=np.sqrt(1.0/n_out), size=filter_shape),
                dtype=theano.config.floatX),
            borrow=True)
        self.b = theano.shared(
            np.asarray(
                np.random.normal(loc=0, scale=1.0, size=(filter_shape[0],)),
                dtype=theano.config.floatX),
            borrow=True)
        self.params = [self.w, self.b]

    def set_inpt(self, inpt, inpt_dropout, mini_batch_size):
        self.inpt = inpt.reshape(self.image_shape)
        conv_out = conv.conv2d(
            input=self.inpt, filters=self.w, filter_shape=self.filter_shape,
            image_shape=self.image_shape)
        pooled_out = pool.pool_2d(
            input=conv_out, ds=self.poolsize, ignore_border=True)
        self.output = self.activation_fn(
            pooled_out + self.b.dimshuffle('x', 0, 'x', 'x'))
        self.output_dropout = self.output # no dropout in the convolutional layers

class FullyConnectedLayer(object):

    def __init__(self, n_in, n_out, activation_fn=sigmoid, p_dropout=0.0):
        self.n_in = n_in
        self.n_out = n_out
        self.activation_fn = activation_fn
        self.p_dropout = p_dropout
        # Initialize weights and biases
        self.w = theano.shared(
            np.asarray(
                np.random.normal(
                    loc=0.0, scale=np.sqrt(1.0/n_out), size=(n_in, n_out)),
                dtype=theano.config.floatX),
            name='w', borrow=True)
        self.b = theano.shared(
            np.asarray(np.random.normal(loc=0.0, scale=1.0, size=(n_out,)),
                       dtype=theano.config.floatX),
            name='b', borrow=True)
        self.params = [self.w, self.b]

    def set_inpt(self, inpt, inpt_dropout, mini_batch_size):
        self.inpt = inpt.reshape((mini_batch_size, self.n_in))
        self.output = self.activation_fn(
            (1-self.p_dropout)*T.dot(self.inpt, self.w) + self.b)
        self.y_out = T.argmax(self.output, axis=1)
        self.inpt_dropout = dropout_layer(
            inpt_dropout.reshape((mini_batch_size, self.n_in)), self.p_dropout)
        self.output_dropout = self.activation_fn(
            T.dot(self.inpt_dropout, self.w) + self.b)

    def accuracy(self, y):
        "Return the accuracy for the mini-batch."
        return T.mean(T.eq(y, self.y_out))

class SoftmaxLayer(object):

    def __init__(self, n_in, n_out, p_dropout=0.0):
        self.n_in = n_in
        self.n_out = n_out
        self.p_dropout = p_dropout
        # Initialize weights and biases
        self.w = theano.shared(
            np.zeros((n_in, n_out), dtype=theano.config.floatX),
            name='w', borrow=True)
        self.b = theano.shared(
            np.zeros((n_out,), dtype=theano.config.floatX),
            name='b', borrow=True)
        self.params = [self.w, self.b]

    def set_inpt(self, inpt, inpt_dropout, mini_batch_size):
        self.inpt = inpt.reshape((mini_batch_size, self.n_in))
        self.output = softmax((1-self.p_dropout)*T.dot(self.inpt, self.w) + self.b)
        self.y_out = T.argmax(self.output, axis=1)
        self.inpt_dropout = dropout_layer(
            inpt_dropout.reshape((mini_batch_size, self.n_in)), self.p_dropout)
        self.output_dropout = softmax(T.dot(self.inpt_dropout, self.w) + self.b)

    def cost(self, net):
        "Return the log-likelihood cost."
        return -T.mean(T.log(self.output_dropout)[T.arange(net.y.shape[0]), net.y])

    def accuracy(self, y):
        "Return the accuracy for the mini-batch."
        return T.mean(T.eq(y, self.y_out))


In [6]:
#### Miscellanea
def size(data):
    "Return the size of the dataset `data`."
    return data[0].get_value(borrow=True).shape[0]

def dropout_layer(layer, p_dropout):
    srng = shared_randomstreams.RandomStreams(
        np.random.RandomState(0).randint(999999))
    mask = srng.binomial(n=1, p=1-p_dropout, size=layer.shape)
    return layer*T.cast(mask, theano.config.floatX)

In [57]:
training_data, validation_data, test_data =load_data_shared()
mini_batch_size = 10
net = Network([         #构建一个只含一层隐含层，包含100个结点
        FullyConnectedLayer(n_in=784, n_out=100),
        SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)
net.SGD(training_data, 60, mini_batch_size, 0.1,
            validation_data, test_data)


Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Epoch 0: validation accuracy 92.51%
This is the best validation accuracy to date.
The corresponding test accuracy is 91.82%
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Epoch 1: validation accuracy 94.54%
This is the best validation accuracy to date.
The corresponding test accuracy is 94.03%
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Epoch 2: validation accuracy 95.64%
This is the best validation accuracy to date.
The corresponding test accuracy is 95.03%
Training mini-batch number 15000
Training mini-batch number 16000
Training mini-batch number 17000
Training mini-batch number 18000
Training mini-

Training mini-batch number 151000
Training mini-batch number 152000
Training mini-batch number 153000
Training mini-batch number 154000
Epoch 30: validation accuracy 97.59%
Training mini-batch number 155000
Training mini-batch number 156000
Training mini-batch number 157000
Training mini-batch number 158000
Training mini-batch number 159000
Epoch 31: validation accuracy 97.63%
This is the best validation accuracy to date.
The corresponding test accuracy is 97.77%
Training mini-batch number 160000
Training mini-batch number 161000
Training mini-batch number 162000
Training mini-batch number 163000
Training mini-batch number 164000
Epoch 32: validation accuracy 97.60%
Training mini-batch number 165000
Training mini-batch number 166000
Training mini-batch number 167000
Training mini-batch number 168000
Training mini-batch number 169000
Epoch 33: validation accuracy 97.60%
Training mini-batch number 170000
Training mini-batch number 171000
Training mini-batch number 172000
Training mini-ba

In [66]:
#加上卷积层后，精确度上升
net = Network([
        ConvPoolLayer(image_shape=(mini_batch_size, 1, 28, 28), 
                      filter_shape=(20, 1, 5, 5), 
                      poolsize=(2, 2)),
        FullyConnectedLayer(n_in=20*12*12, n_out=100),
        SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)
net.SGD(training_data, 60, mini_batch_size, 0.1, 
            validation_data, test_data)   

/home/keqingtian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.


Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Epoch 0: validation accuracy 93.96%
This is the best validation accuracy to date.
The corresponding test accuracy is 93.17%
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Epoch 1: validation accuracy 96.33%
This is the best validation accuracy to date.
The corresponding test accuracy is 95.76%
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Epoch 2: validation accuracy 97.25%
This is the best validation accuracy to date.
The corresponding test accuracy is 96.79%
Training mini-batch number 15000
Training mini-batch number 16000
Training mini-batch number 17000
Training mini-batch number 18000
Training mini-

Training mini-batch number 161000
Training mini-batch number 162000
Training mini-batch number 163000
Training mini-batch number 164000
Epoch 32: validation accuracy 98.66%
Training mini-batch number 165000
Training mini-batch number 166000
Training mini-batch number 167000
Training mini-batch number 168000
Training mini-batch number 169000
Epoch 33: validation accuracy 98.67%
Training mini-batch number 170000
Training mini-batch number 171000
Training mini-batch number 172000
Training mini-batch number 173000
Training mini-batch number 174000
Epoch 34: validation accuracy 98.67%
Training mini-batch number 175000
Training mini-batch number 176000
Training mini-batch number 177000
Training mini-batch number 178000
Training mini-batch number 179000
Epoch 35: validation accuracy 98.68%
This is the best validation accuracy to date.
The corresponding test accuracy is 98.58%
Training mini-batch number 180000
Training mini-batch number 181000
Training mini-batch number 182000
Training mini-ba

In [67]:
#添加一个卷积-池化层，准确率再次提高
net = Network([
        ConvPoolLayer(image_shape=(mini_batch_size, 1, 28, 28), 
                      filter_shape=(20, 1, 5, 5), 
                      poolsize=(2, 2)),
        ConvPoolLayer(image_shape=(mini_batch_size, 20, 12, 12), 
                      filter_shape=(40, 20, 5, 5), 
                      poolsize=(2, 2)),
        FullyConnectedLayer(n_in=40*4*4, n_out=100),
        SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)
net.SGD(training_data, 60, mini_batch_size, 0.1, 
            validation_data, test_data)        

/home/keqingtian/anaconda3/lib/python3.7/site-packages/theano/tensor/nnet/conv.py:98: UserWarning: theano.tensor.nnet.conv.conv2d is deprecated. Use theano.tensor.nnet.conv2d instead.
  warnings.warn("theano.tensor.nnet.conv.conv2d is deprecated."
/home/keqingtian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.


Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Epoch 0: validation accuracy 91.57%
This is the best validation accuracy to date.
The corresponding test accuracy is 91.29%
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Epoch 1: validation accuracy 96.30%
This is the best validation accuracy to date.
The corresponding test accuracy is 96.15%
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Epoch 2: validation accuracy 97.27%
This is the best validation accuracy to date.
The corresponding test accuracy is 97.03%
Training mini-batch number 15000
Training mini-batch number 16000
Training mini-batch number 17000
Training mini-batch number 18000
Training mini-

Training mini-batch number 151000
Training mini-batch number 152000
Training mini-batch number 153000
Training mini-batch number 154000
Epoch 30: validation accuracy 98.84%
Training mini-batch number 155000
Training mini-batch number 156000
Training mini-batch number 157000
Training mini-batch number 158000
Training mini-batch number 159000
Epoch 31: validation accuracy 98.85%
Training mini-batch number 160000
Training mini-batch number 161000
Training mini-batch number 162000
Training mini-batch number 163000
Training mini-batch number 164000
Epoch 32: validation accuracy 98.85%
Training mini-batch number 165000
Training mini-batch number 166000
Training mini-batch number 167000
Training mini-batch number 168000
Training mini-batch number 169000
Epoch 33: validation accuracy 98.83%
Training mini-batch number 170000
Training mini-batch number 171000
Training mini-batch number 172000
Training mini-batch number 173000
Training mini-batch number 174000
Epoch 34: validation accuracy 98.83%

In [68]:
#激活函数换成整流线性单元Relu,学习率0.03，L2规则化参数lamda=0.1
net = Network([
        ConvPoolLayer(image_shape=(mini_batch_size, 1, 28, 28), 
                      filter_shape=(20, 1, 5, 5), 
                      poolsize=(2, 2), 
                      activation_fn=ReLU),
        ConvPoolLayer(image_shape=(mini_batch_size, 20, 12, 12), 
                      filter_shape=(40, 20, 5, 5), 
                      poolsize=(2, 2), 
                      activation_fn=ReLU),
        FullyConnectedLayer(n_in=40*4*4, n_out=100, activation_fn=ReLU),
        SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)
net.SGD(training_data, 60, mini_batch_size, 0.03, 
            validation_data, test_data, lmbda=0.1)

/home/keqingtian/anaconda3/lib/python3.7/site-packages/theano/tensor/nnet/conv.py:98: UserWarning: theano.tensor.nnet.conv.conv2d is deprecated. Use theano.tensor.nnet.conv2d instead.
  warnings.warn("theano.tensor.nnet.conv.conv2d is deprecated."
/home/keqingtian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.


Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Epoch 0: validation accuracy 97.57%
This is the best validation accuracy to date.
The corresponding test accuracy is 97.19%
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Epoch 1: validation accuracy 97.96%
This is the best validation accuracy to date.
The corresponding test accuracy is 97.40%
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Epoch 2: validation accuracy 98.40%
This is the best validation accuracy to date.
The corresponding test accuracy is 98.15%
Training mini-batch number 15000
Training mini-batch number 16000
Training mini-batch number 17000
Training mini-batch number 18000
Training mini-

The corresponding test accuracy is 99.10%
Training mini-batch number 160000
Training mini-batch number 161000
Training mini-batch number 162000
Training mini-batch number 163000
Training mini-batch number 164000
Epoch 32: validation accuracy 99.08%
This is the best validation accuracy to date.
The corresponding test accuracy is 99.10%
Training mini-batch number 165000
Training mini-batch number 166000
Training mini-batch number 167000
Training mini-batch number 168000
Training mini-batch number 169000
Epoch 33: validation accuracy 99.08%
This is the best validation accuracy to date.
The corresponding test accuracy is 99.10%
Training mini-batch number 170000
Training mini-batch number 171000
Training mini-batch number 172000
Training mini-batch number 173000
Training mini-batch number 174000
Epoch 34: validation accuracy 99.08%
This is the best validation accuracy to date.
The corresponding test accuracy is 99.10%
Training mini-batch number 175000
Training mini-batch number 176000
Train

In [78]:
#数据增强，扩充数据集，原文件在expand_mnist.py中，5000张变为25000张
# Standard library
import pickle
import gzip
import os.path
import random

# Third-party libraries
import numpy as np

print("Expanding the MNIST training set")

f = gzip.open("/home/keqingtian/neural-networks-and-deep-learning/data/mnist.pkl.gz", 'rb')
training_data, validation_data, test_data = pickle.load(f,encoding="bytes")
f.close()
expanded_training_pairs = []    #初始化
j = 0 # counter
for x, y in zip(training_data[0], training_data[1]):
    expanded_training_pairs.append((x, y))
    image = np.reshape(x, (-1, 28))
    j += 1
    if j % 1000 == 0: print("Expanding image number", j)
   # iterate over data telling us the details of how to
   # do the displacement
    for d, axis, index_position, index in [
                (1,  0, "first", 0),
                (-1, 0, "first", 27),
                (1,  1, "last",  0),
                (-1, 1, "last",  27)]:
        new_img = np.roll(image, d, axis)   #从左到右
        if index_position == "first":    
            new_img[index, :] = np.zeros(28)
        else: 
            new_img[:, index] = np.zeros(28)
        expanded_training_pairs.append((np.reshape(new_img, 784), y))
random.shuffle(expanded_training_pairs)
expanded_training_data = [list(d) for d in zip(*expanded_training_pairs)]
print("Saving expanded data. This may take a few minutes.")
f = gzip.open("/home/keqingtian/neural-networks-and-deep-learning/data/mnist_expanded.pkl.gz", "w")
pickle.dump((expanded_training_data, validation_data, test_data), f)
f.close()

Expanding the MNIST training set
Expanding image number 1000
Expanding image number 2000
Expanding image number 3000
Expanding image number 4000
Expanding image number 5000
Expanding image number 6000
Expanding image number 7000
Expanding image number 8000
Expanding image number 9000
Expanding image number 10000
Expanding image number 11000
Expanding image number 12000
Expanding image number 13000
Expanding image number 14000
Expanding image number 15000
Expanding image number 16000
Expanding image number 17000
Expanding image number 18000
Expanding image number 19000
Expanding image number 20000
Expanding image number 21000
Expanding image number 22000
Expanding image number 23000
Expanding image number 24000
Expanding image number 25000
Expanding image number 26000
Expanding image number 27000
Expanding image number 28000
Expanding image number 29000
Expanding image number 30000
Expanding image number 31000
Expanding image number 32000
Expanding image number 33000
Expanding image num

In [7]:
#用扩充的数据集训练，减少过拟合，60 epochs
expanded_training_data, validation_data, test_data = load_data_shared(
        "/home/keqingtian/neural-networks-and-deep-learning/data/mnist_expanded.pkl.gz")
mini_batch_size = 10
net = Network([
        ConvPoolLayer(image_shape=(mini_batch_size, 1, 28, 28), 
                      filter_shape=(20, 1, 5, 5), 
                      poolsize=(2, 2), 
                      activation_fn=ReLU),
        ConvPoolLayer(image_shape=(mini_batch_size, 20, 12, 12), 
                      filter_shape=(40, 20, 5, 5), 
                      poolsize=(2, 2), 
                      activation_fn=ReLU),
        FullyConnectedLayer(n_in=40*4*4, n_out=100, activation_fn=ReLU),
        SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)
net.SGD(expanded_training_data, 60, mini_batch_size, 0.03, 
            validation_data, test_data, lmbda=0.1)

/home/keqingtian/anaconda3/lib/python3.7/site-packages/theano/tensor/nnet/conv.py:98: UserWarning: theano.tensor.nnet.conv.conv2d is deprecated. Use theano.tensor.nnet.conv2d instead.
  warnings.warn("theano.tensor.nnet.conv.conv2d is deprecated."
/home/keqingtian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.


Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Training mini-batch number 15000
Training mini-batch number 16000
Training mini-batch number 17000
Training mini-batch number 18000
Training mini-batch number 19000
Training mini-batch number 20000
Training mini-batch number 21000
Training mini-batch number 22000
Training mini-batch number 23000
Training mini-batch number 24000
Epoch 0: validation accuracy 98.80%
This is the best validation accuracy to date.
The corresponding test accuracy is 98.92%
Training mini-batch number 25000
Training mini-batch number 2600

Training mini-batch number 227000
Training mini-batch number 228000
Training mini-batch number 229000
Training mini-batch number 230000
Training mini-batch number 231000
Training mini-batch number 232000
Training mini-batch number 233000
Training mini-batch number 234000
Training mini-batch number 235000
Training mini-batch number 236000
Training mini-batch number 237000
Training mini-batch number 238000
Training mini-batch number 239000
Training mini-batch number 240000
Training mini-batch number 241000
Training mini-batch number 242000
Training mini-batch number 243000
Training mini-batch number 244000
Training mini-batch number 245000
Training mini-batch number 246000
Training mini-batch number 247000
Training mini-batch number 248000
Training mini-batch number 249000
Epoch 9: validation accuracy 99.25%
Training mini-batch number 250000
Training mini-batch number 251000
Training mini-batch number 252000
Training mini-batch number 253000
Training mini-batch number 254000
Training min

The corresponding test accuracy is 99.39%
Training mini-batch number 450000
Training mini-batch number 451000
Training mini-batch number 452000
Training mini-batch number 453000
Training mini-batch number 454000
Training mini-batch number 455000
Training mini-batch number 456000
Training mini-batch number 457000
Training mini-batch number 458000
Training mini-batch number 459000
Training mini-batch number 460000
Training mini-batch number 461000
Training mini-batch number 462000
Training mini-batch number 463000
Training mini-batch number 464000
Training mini-batch number 465000
Training mini-batch number 466000
Training mini-batch number 467000
Training mini-batch number 468000
Training mini-batch number 469000
Training mini-batch number 470000
Training mini-batch number 471000
Training mini-batch number 472000
Training mini-batch number 473000
Training mini-batch number 474000
Epoch 18: validation accuracy 99.38%
Training mini-batch number 475000
Training mini-batch number 476000
Tra

Training mini-batch number 678000
Training mini-batch number 679000
Training mini-batch number 680000
Training mini-batch number 681000
Training mini-batch number 682000
Training mini-batch number 683000
Training mini-batch number 684000
Training mini-batch number 685000
Training mini-batch number 686000
Training mini-batch number 687000
Training mini-batch number 688000
Training mini-batch number 689000
Training mini-batch number 690000
Training mini-batch number 691000
Training mini-batch number 692000
Training mini-batch number 693000
Training mini-batch number 694000
Training mini-batch number 695000
Training mini-batch number 696000
Training mini-batch number 697000
Training mini-batch number 698000
Training mini-batch number 699000
Epoch 27: validation accuracy 99.40%
Training mini-batch number 700000
Training mini-batch number 701000
Training mini-batch number 702000
Training mini-batch number 703000
Training mini-batch number 704000
Training mini-batch number 705000
Training mi

Training mini-batch number 910000
Training mini-batch number 911000
Training mini-batch number 912000
Training mini-batch number 913000
Training mini-batch number 914000
Training mini-batch number 915000
Training mini-batch number 916000
Training mini-batch number 917000
Training mini-batch number 918000
Training mini-batch number 919000
Training mini-batch number 920000
Training mini-batch number 921000
Training mini-batch number 922000
Training mini-batch number 923000
Training mini-batch number 924000
Epoch 36: validation accuracy 99.39%
Training mini-batch number 925000
Training mini-batch number 926000
Training mini-batch number 927000
Training mini-batch number 928000
Training mini-batch number 929000
Training mini-batch number 930000
Training mini-batch number 931000
Training mini-batch number 932000
Training mini-batch number 933000
Training mini-batch number 934000
Training mini-batch number 935000
Training mini-batch number 936000
Training mini-batch number 937000
Training mi

Training mini-batch number 1126000
Training mini-batch number 1127000
Training mini-batch number 1128000
Training mini-batch number 1129000
Training mini-batch number 1130000
Training mini-batch number 1131000
Training mini-batch number 1132000
Training mini-batch number 1133000
Training mini-batch number 1134000
Training mini-batch number 1135000
Training mini-batch number 1136000
Training mini-batch number 1137000
Training mini-batch number 1138000
Training mini-batch number 1139000
Training mini-batch number 1140000
Training mini-batch number 1141000
Training mini-batch number 1142000
Training mini-batch number 1143000
Training mini-batch number 1144000
Training mini-batch number 1145000
Training mini-batch number 1146000
Training mini-batch number 1147000
Training mini-batch number 1148000
Training mini-batch number 1149000
Epoch 45: validation accuracy 99.44%
Training mini-batch number 1150000
Training mini-batch number 1151000
Training mini-batch number 1152000
Training mini-batc

Training mini-batch number 1351000
Training mini-batch number 1352000
Training mini-batch number 1353000
Training mini-batch number 1354000
Training mini-batch number 1355000
Training mini-batch number 1356000
Training mini-batch number 1357000
Training mini-batch number 1358000
Training mini-batch number 1359000
Training mini-batch number 1360000
Training mini-batch number 1361000
Training mini-batch number 1362000
Training mini-batch number 1363000
Training mini-batch number 1364000
Training mini-batch number 1365000
Training mini-batch number 1366000
Training mini-batch number 1367000
Training mini-batch number 1368000
Training mini-batch number 1369000
Training mini-batch number 1370000
Training mini-batch number 1371000
Training mini-batch number 1372000
Training mini-batch number 1373000
Training mini-batch number 1374000
Epoch 54: validation accuracy 99.43%
Training mini-batch number 1375000
Training mini-batch number 1376000
Training mini-batch number 1377000
Training mini-batc

In [8]:
#当再加上一个全连接层后，精度达到99.43%
net = Network([
        ConvPoolLayer(image_shape=(mini_batch_size, 1, 28, 28), 
                      filter_shape=(20, 1, 5, 5), 
                      poolsize=(2, 2), 
                      activation_fn=ReLU),
        ConvPoolLayer(image_shape=(mini_batch_size, 20, 12, 12), 
                      filter_shape=(40, 20, 5, 5), 
                      poolsize=(2, 2), 
                      activation_fn=ReLU),
        FullyConnectedLayer(n_in=40*4*4, n_out=100, activation_fn=ReLU),
        FullyConnectedLayer(n_in=100, n_out=100, activation_fn=ReLU),
        SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)
net.SGD(expanded_training_data, 60, mini_batch_size, 0.03, 
            validation_data, test_data, lmbda=0.1)


/home/keqingtian/anaconda3/lib/python3.7/site-packages/theano/tensor/nnet/conv.py:98: UserWarning: theano.tensor.nnet.conv.conv2d is deprecated. Use theano.tensor.nnet.conv2d instead.
  warnings.warn("theano.tensor.nnet.conv.conv2d is deprecated."
/home/keqingtian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.


Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Training mini-batch number 15000
Training mini-batch number 16000
Training mini-batch number 17000
Training mini-batch number 18000
Training mini-batch number 19000
Training mini-batch number 20000
Training mini-batch number 21000
Training mini-batch number 22000
Training mini-batch number 23000
Training mini-batch number 24000
Epoch 0: validation accuracy 98.96%
This is the best validation accuracy to date.
The corresponding test accuracy is 98.98%
Training mini-batch number 25000
Training mini-batch number 2600

The corresponding test accuracy is 99.41%
Training mini-batch number 225000
Training mini-batch number 226000
Training mini-batch number 227000
Training mini-batch number 228000
Training mini-batch number 229000
Training mini-batch number 230000
Training mini-batch number 231000
Training mini-batch number 232000
Training mini-batch number 233000
Training mini-batch number 234000
Training mini-batch number 235000
Training mini-batch number 236000
Training mini-batch number 237000
Training mini-batch number 238000
Training mini-batch number 239000
Training mini-batch number 240000
Training mini-batch number 241000
Training mini-batch number 242000
Training mini-batch number 243000
Training mini-batch number 244000
Training mini-batch number 245000
Training mini-batch number 246000
Training mini-batch number 247000
Training mini-batch number 248000
Training mini-batch number 249000
Epoch 9: validation accuracy 99.17%
Training mini-batch number 250000
Training mini-batch number 251000
Trai

Training mini-batch number 455000
Training mini-batch number 456000
Training mini-batch number 457000
Training mini-batch number 458000
Training mini-batch number 459000
Training mini-batch number 460000
Training mini-batch number 461000
Training mini-batch number 462000
Training mini-batch number 463000
Training mini-batch number 464000
Training mini-batch number 465000
Training mini-batch number 466000
Training mini-batch number 467000
Training mini-batch number 468000
Training mini-batch number 469000
Training mini-batch number 470000
Training mini-batch number 471000
Training mini-batch number 472000
Training mini-batch number 473000
Training mini-batch number 474000
Epoch 18: validation accuracy 99.27%
Training mini-batch number 475000
Training mini-batch number 476000
Training mini-batch number 477000
Training mini-batch number 478000
Training mini-batch number 479000
Training mini-batch number 480000
Training mini-batch number 481000
Training mini-batch number 482000
Training mi

Training mini-batch number 687000
Training mini-batch number 688000
Training mini-batch number 689000
Training mini-batch number 690000
Training mini-batch number 691000
Training mini-batch number 692000
Training mini-batch number 693000
Training mini-batch number 694000
Training mini-batch number 695000
Training mini-batch number 696000
Training mini-batch number 697000
Training mini-batch number 698000
Training mini-batch number 699000
Epoch 27: validation accuracy 99.36%
This is the best validation accuracy to date.
The corresponding test accuracy is 99.39%
Training mini-batch number 700000
Training mini-batch number 701000
Training mini-batch number 702000
Training mini-batch number 703000
Training mini-batch number 704000
Training mini-batch number 705000
Training mini-batch number 706000
Training mini-batch number 707000
Training mini-batch number 708000
Training mini-batch number 709000
Training mini-batch number 710000
Training mini-batch number 711000
Training mini-batch numbe

Training mini-batch number 913000
Training mini-batch number 914000
Training mini-batch number 915000
Training mini-batch number 916000
Training mini-batch number 917000
Training mini-batch number 918000
Training mini-batch number 919000
Training mini-batch number 920000
Training mini-batch number 921000
Training mini-batch number 922000
Training mini-batch number 923000
Training mini-batch number 924000
Epoch 36: validation accuracy 99.26%
Training mini-batch number 925000
Training mini-batch number 926000
Training mini-batch number 927000
Training mini-batch number 928000
Training mini-batch number 929000
Training mini-batch number 930000
Training mini-batch number 931000
Training mini-batch number 932000
Training mini-batch number 933000
Training mini-batch number 934000
Training mini-batch number 935000
Training mini-batch number 936000
Training mini-batch number 937000
Training mini-batch number 938000
Training mini-batch number 939000
Training mini-batch number 940000
Training mi

Training mini-batch number 1141000
Training mini-batch number 1142000
Training mini-batch number 1143000
Training mini-batch number 1144000
Training mini-batch number 1145000
Training mini-batch number 1146000
Training mini-batch number 1147000
Training mini-batch number 1148000
Training mini-batch number 1149000
Epoch 45: validation accuracy 99.35%
Training mini-batch number 1150000
Training mini-batch number 1151000
Training mini-batch number 1152000
Training mini-batch number 1153000
Training mini-batch number 1154000
Training mini-batch number 1155000
Training mini-batch number 1156000
Training mini-batch number 1157000
Training mini-batch number 1158000
Training mini-batch number 1159000
Training mini-batch number 1160000
Training mini-batch number 1161000
Training mini-batch number 1162000
Training mini-batch number 1163000
Training mini-batch number 1164000
Training mini-batch number 1165000
Training mini-batch number 1166000
Training mini-batch number 1167000
Training mini-batc

Training mini-batch number 1366000
Training mini-batch number 1367000
Training mini-batch number 1368000
Training mini-batch number 1369000
Training mini-batch number 1370000
Training mini-batch number 1371000
Training mini-batch number 1372000
Training mini-batch number 1373000
Training mini-batch number 1374000
Epoch 54: validation accuracy 99.36%
Training mini-batch number 1375000
Training mini-batch number 1376000
Training mini-batch number 1377000
Training mini-batch number 1378000
Training mini-batch number 1379000
Training mini-batch number 1380000
Training mini-batch number 1381000
Training mini-batch number 1382000
Training mini-batch number 1383000
Training mini-batch number 1384000
Training mini-batch number 1385000
Training mini-batch number 1386000
Training mini-batch number 1387000
Training mini-batch number 1388000
Training mini-batch number 1389000
Training mini-batch number 1390000
Training mini-batch number 1391000
Training mini-batch number 1392000
Training mini-batc

In [9]:
#进一步正则化，在最后一层引入dropout,精度达到99.6%
net = Network([
        ConvPoolLayer(image_shape=(mini_batch_size, 1, 28, 28), 
                      filter_shape=(20, 1, 5, 5), 
                      poolsize=(2, 2), 
                      activation_fn=ReLU),
        ConvPoolLayer(image_shape=(mini_batch_size, 20, 12, 12), 
                      filter_shape=(40, 20, 5, 5), 
                      poolsize=(2, 2), 
                      activation_fn=ReLU),
        FullyConnectedLayer(
            n_in=40*4*4, n_out=1000, activation_fn=ReLU, p_dropout=0.5),
        FullyConnectedLayer(
            n_in=1000, n_out=1000, activation_fn=ReLU, p_dropout=0.5),
        SoftmaxLayer(n_in=1000, n_out=10, p_dropout=0.5)], 
        mini_batch_size)
net.SGD(expanded_training_data, 40, mini_batch_size, 0.03, 
            validation_data, test_data)

/home/keqingtian/anaconda3/lib/python3.7/site-packages/theano/tensor/nnet/conv.py:98: UserWarning: theano.tensor.nnet.conv.conv2d is deprecated. Use theano.tensor.nnet.conv2d instead.
  warnings.warn("theano.tensor.nnet.conv.conv2d is deprecated."
/home/keqingtian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.


Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Training mini-batch number 15000
Training mini-batch number 16000
Training mini-batch number 17000
Training mini-batch number 18000
Training mini-batch number 19000
Training mini-batch number 20000
Training mini-batch number 21000
Training mini-batch number 22000
Training mini-batch number 23000
Training mini-batch number 24000
Epoch 0: validation accuracy 98.67%
This is the best validation accuracy to date.
The corresponding test accuracy is 98.81%
Training mini-batch number 25000
Training mini-batch number 2600

Training mini-batch number 216000
Training mini-batch number 217000
Training mini-batch number 218000
Training mini-batch number 219000
Training mini-batch number 220000
Training mini-batch number 221000
Training mini-batch number 222000
Training mini-batch number 223000
Training mini-batch number 224000
Epoch 8: validation accuracy 99.48%
This is the best validation accuracy to date.
The corresponding test accuracy is 99.56%
Training mini-batch number 225000
Training mini-batch number 226000
Training mini-batch number 227000
Training mini-batch number 228000
Training mini-batch number 229000
Training mini-batch number 230000
Training mini-batch number 231000
Training mini-batch number 232000
Training mini-batch number 233000
Training mini-batch number 234000
Training mini-batch number 235000
Training mini-batch number 236000
Training mini-batch number 237000
Training mini-batch number 238000
Training mini-batch number 239000
Training mini-batch number 240000
Training mini-batch number

Training mini-batch number 437000
Training mini-batch number 438000
Training mini-batch number 439000
Training mini-batch number 440000
Training mini-batch number 441000
Training mini-batch number 442000
Training mini-batch number 443000
Training mini-batch number 444000
Training mini-batch number 445000
Training mini-batch number 446000
Training mini-batch number 447000
Training mini-batch number 448000
Training mini-batch number 449000
Epoch 17: validation accuracy 99.60%
This is the best validation accuracy to date.
The corresponding test accuracy is 99.58%
Training mini-batch number 450000
Training mini-batch number 451000
Training mini-batch number 452000
Training mini-batch number 453000
Training mini-batch number 454000
Training mini-batch number 455000
Training mini-batch number 456000
Training mini-batch number 457000
Training mini-batch number 458000
Training mini-batch number 459000
Training mini-batch number 460000
Training mini-batch number 461000
Training mini-batch numbe

Training mini-batch number 658000
Training mini-batch number 659000
Training mini-batch number 660000
Training mini-batch number 661000
Training mini-batch number 662000
Training mini-batch number 663000
Training mini-batch number 664000
Training mini-batch number 665000
Training mini-batch number 666000
Training mini-batch number 667000
Training mini-batch number 668000
Training mini-batch number 669000
Training mini-batch number 670000
Training mini-batch number 671000
Training mini-batch number 672000
Training mini-batch number 673000
Training mini-batch number 674000
Epoch 26: validation accuracy 99.61%
Training mini-batch number 675000
Training mini-batch number 676000
Training mini-batch number 677000
Training mini-batch number 678000
Training mini-batch number 679000
Training mini-batch number 680000
Training mini-batch number 681000
Training mini-batch number 682000
Training mini-batch number 683000
Training mini-batch number 684000
Training mini-batch number 685000
Training mi

Training mini-batch number 887000
Training mini-batch number 888000
Training mini-batch number 889000
Training mini-batch number 890000
Training mini-batch number 891000
Training mini-batch number 892000
Training mini-batch number 893000
Training mini-batch number 894000
Training mini-batch number 895000
Training mini-batch number 896000
Training mini-batch number 897000
Training mini-batch number 898000
Training mini-batch number 899000
Epoch 35: validation accuracy 99.60%
Training mini-batch number 900000
Training mini-batch number 901000
Training mini-batch number 902000
Training mini-batch number 903000
Training mini-batch number 904000
Training mini-batch number 905000
Training mini-batch number 906000
Training mini-batch number 907000
Training mini-batch number 908000
Training mini-batch number 909000
Training mini-batch number 910000
Training mini-batch number 911000
Training mini-batch number 912000
Training mini-batch number 913000
Training mini-batch number 914000
Training mi